In [1]:
# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Ignore useless warnings (see SciPy issue #5998)
import warnings
warnings.filterwarnings(action="ignore", message="^internal gelsd")

import tarfile
from six.moves import urllib

In [5]:
# train_users.csv - the training set of users
pd.read_csv('train_users_2.csv').head(5)

id date_account_created  timestamp_first_active date_first_booking  \
0  gxn3p5htnn           2010-06-28          20090319043255                NaN   
1  820tgsjxq7           2011-05-25          20090523174809                NaN   
2  4ft3gnwmtx           2010-09-28          20090609231247         2010-08-02   
3  bjjt8pjhuk           2011-12-05          20091031060129         2012-09-08   
4  87mebub9p4           2010-09-14          20091208061105         2010-02-18   

      gender   age signup_method  signup_flow language affiliate_channel  \
0  -unknown-   NaN      facebook            0       en            direct   
1       MALE  38.0      facebook            0       en               seo   
2     FEMALE  56.0         basic            3       en            direct   
3     FEMALE  42.0      facebook            0       en            direct   
4  -unknown-  41.0         basic            0       en            direct   

  affiliate_provider first_affiliate_tracked signup_app first_device_type  \
0             direct               untracked        Web       Mac Desktop   
1             google               untracked        Web       Mac Desktop   
2             direct               untracked        Web   Windows Desktop   
3             direct               untracked        Web       Mac Desktop   
4             direct               untracked        Web       Mac Desktop   

  first_browser country_destination  
0        Chrome                 NDF  
1        Chrome                 NDF  
2            IE                  US  
3       Firefox               other  
4        Chrome                  US

In [6]:
# test_users.csv - the test set of users
# id: user id
# date_account_created: the date of account creation
# timestamp_first_active: timestamp of the first activity, note that it can be earlier than date_account_created or date_first_booking because a user can search before signing up
# date_first_booking: date of first booking
# gender
# age
# signup_method
# signup_flow: the page a user came to signup up from
# language: international language preference
# affiliate_channel: what kind of paid marketing
# affiliate_provider: where the marketing is e.g. google, craigslist, other
# first_affiliate_tracked: whats the first marketing the user interacted with before the signing up
# signup_app
# first_device_type
# first_browser
# country_destination: this is the target variable you are to predict
pd.read_csv('test_users.csv').head(5)

id date_account_created  timestamp_first_active  \
0  5uwns89zht           2014-07-01          20140701000006   
1  jtl0dijy2j           2014-07-01          20140701000051   
2  xx0ulgorjt           2014-07-01          20140701000148   
3  6c6puo6ix0           2014-07-01          20140701000215   
4  czqhjk3yfe           2014-07-01          20140701000305   

   date_first_booking     gender   age signup_method  signup_flow language  \
0                 NaN     FEMALE  35.0      facebook            0       en   
1                 NaN  -unknown-   NaN         basic            0       en   
2                 NaN  -unknown-   NaN         basic            0       en   
3                 NaN  -unknown-   NaN         basic            0       en   
4                 NaN  -unknown-   NaN         basic            0       en   

  affiliate_channel affiliate_provider first_affiliate_tracked signup_app  \
0            direct             direct               untracked      Moweb   
1            direct             direct               untracked      Moweb   
2            direct             direct                  linked        Web   
3            direct             direct                  linked        Web   
4            direct             direct               untracked        Web   

  first_device_type  first_browser  
0            iPhone  Mobile Safari  
1            iPhone  Mobile Safari  
2   Windows Desktop         Chrome  
3   Windows Desktop             IE  
4       Mac Desktop         Safari

In [8]:
# sessions.csv - web sessions log for users
# user_id: to be joined with the column 'id' in users table
# action
# action_type
# action_detail
# device_type
# secs_elapsed
pd.read_csv('sessions.csv').head(10)

user_id          action action_type            action_detail  \
0  d1mm9tcy42          lookup         NaN                      NaN   
1  d1mm9tcy42  search_results       click      view_search_results   
2  d1mm9tcy42          lookup         NaN                      NaN   
3  d1mm9tcy42  search_results       click      view_search_results   
4  d1mm9tcy42          lookup         NaN                      NaN   
5  d1mm9tcy42  search_results       click      view_search_results   
6  d1mm9tcy42          lookup         NaN                      NaN   
7  d1mm9tcy42     personalize        data  wishlist_content_update   
8  d1mm9tcy42           index        view      view_search_results   
9  d1mm9tcy42          lookup         NaN                      NaN   

       device_type  secs_elapsed  
0  Windows Desktop         319.0  
1  Windows Desktop       67753.0  
2  Windows Desktop         301.0  
3  Windows Desktop       22141.0  
4  Windows Desktop         435.0  
5  Windows Desktop        7703.0  
6  Windows Desktop         115.0  
7  Windows Desktop         831.0  
8  Windows Desktop       20842.0  
9  Windows Desktop         683.0

In [9]:
# countries.csv - summary statistics of destination countries in this dataset and their locations
pd.read_csv('countries.csv')

country_destination  lat_destination  lng_destination  distance_km  \
0                  AU       -26.853388       133.275160   15297.7440   
1                  CA        62.393303       -96.818146    2828.1333   
2                  DE        51.165707        10.452764    7879.5680   
3                  ES        39.896027        -2.487694    7730.7240   
4                  FR        46.232193         2.209667    7682.9450   
5                  GB        54.633220        -3.432277    6883.6590   
6                  IT        41.873990        12.564167    8636.6310   
7                  NL        52.133057         5.295250    7524.3203   
8                  PT        39.553444        -7.839319    7355.2534   
9                  US        36.966427       -95.844030       0.0000   

   destination_km2 destination_language   language_levenshtein_distance  
0        7741220.0                   eng                           0.00  
1        9984670.0                   eng                           0.00  
2         357022.0                   deu                          72.61  
3         505370.0                   spa                          92.25  
4         643801.0                   fra                          92.06  
5         243610.0                   eng                           0.00  
6         301340.0                   ita                          89.40  
7          41543.0                   nld                          63.22  
8          92090.0                   por                          95.45  
9        9826675.0                   eng                           0.00

In [2]:
# age_gender_bkts.csv - summary statistics of users' age group, gender, country of destination
pd.read_csv('age_gender_bkts.csv')

age_bucket country_destination  gender  population_in_thousands    year
0         100+                  AU    male                      1.0  2015.0
1        95-99                  AU    male                      9.0  2015.0
2        90-94                  AU    male                     47.0  2015.0
3        85-89                  AU    male                    118.0  2015.0
4        80-84                  AU    male                    199.0  2015.0
..         ...                 ...     ...                      ...     ...
415      95-99                  US    male                    115.0  2015.0
416      90-94                  US    male                    541.0  2015.0
417      15-19                  US  female                  10570.0  2015.0
418      85-89                  US    male                   1441.0  2015.0
419      80-84                  US    male                   2442.0  2015.0

[420 rows x 5 columns]